In [1]:
import geopandas as gpd
import pandas as pd
import requests,zipfile,io
import os
from src.utils.functions import *
import folium
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
from rasterio.features import shapes

## Download and load Roads Data 

In [2]:
continent_osm = 'africa'
country_full = 'morocco'
country_osm_url = 'https://download.geofabrik.de/%s/%s-latest-free.shp.zip'%(continent_osm,country_full)
data_path = 'Data/'

In [3]:
path_roads = os.path.join(data_path,'Roads','gis_osm_roads_free_1.shp')
        
# Download roads
download_and_extract_roads(country_osm_url, data_path, path_roads)
            
# Read and map roads in categories   
roads_shp_path = os.path.join(path_roads)
load_country = gpd.read_file(roads_shp_path)
uniq = load_country['fclass'].value_counts()
uniq = list(uniq[uniq > 20].index)
uniq.extend(['primary','secondary','trunk','motorway'])
uniq = list(set(uniq))
load_country = load_country[load_country['fclass'].isin(uniq)]
load_country_tot = map_roads(load_country)

# Create Morocco boundary map
morocco_shp = gpd.read_parquet(os.path.join(data_path,'boundaries_morocco.parquet'))

# Filter roads by category
load_country_tot = load_country_tot.to_crs(epsg=26192)
roads_primary_secondary = load_country_tot[load_country_tot.roads.isin(['primary','secondary'])]
roads_tertiary = load_country_tot[load_country_tot.roads.isin(['tertiary'])]

# Select primary, secondary and tertiary roads
roads_pst = load_country[load_country.roads.isin(['primary','secondary','tertiary'])]
roads_pst = roads_pst.rename({"roads":"type"},axis=1)

✅ Roads shapefile downloaded and extracted successfully.


## Case 1: Primary and Secondary roads

In [4]:
sample_points_ps = sample_points_within_roads(roads_primary_secondary)
sample_points_ps = gpd.sjoin(sample_points_ps,morocco_shp,predicate="within")
reachable_areas_ps = compute_isochrones(sample_points_ps,distance=2000,base_url='http://localhost:8080/ors')
missing_areas = morocco_shp.overlay(reachable_areas_ps, how='difference', keep_geom_type=None, make_valid=True)
result_ps = calculate_population(morocco_shp, missing_areas)

Processed 1000 points...
Processed 2000 points...
Processed 3000 points...
Processed 4000 points...
Processed 5000 points...
Processed 6000 points...
Processed 7000 points...
Processed 8000 points...
Processed 9000 points...
Processed 10000 points...
Processed 11000 points...
Processed 12000 points...
Processed 13000 points...
Processed 14000 points...
Processed 15000 points...
Processed 16000 points...
Processed 17000 points...
Processed 18000 points...
Processed 19000 points...
Processed 20000 points...
Processed 21000 points...
Processed 22000 points...
Processed 23000 points...
Processed 24000 points...
Processed 25000 points...
Processed 26000 points...
Processed 27000 points...
Processed 28000 points...
Processed 29000 points...
Processed 30000 points...
Processed 31000 points...
Failed point 31944: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2025-09-30T14:28:56Z', 'graph_version': '3', 'graph_date': '2025-11-1

In [5]:
result_ps

{'pop_total': 38043584.3125, 'pop_excluded': 17392219.125}

## Case 2 : Tertiary Roads

In [6]:
sample_points_t = sample_points_within_roads(roads_tertiary)
sample_points_t = gpd.sjoin(sample_points_t,morocco_shp,predicate="within")
reachable_areas_t = compute_isochrones(sample_points_t,distance=2000,base_url='http://localhost:8080/ors')

reachable_areas_pst = pd.concat([reachable_areas_ps,reachable_areas_t]).dissolve()
missing_areas_pst = morocco_shp.overlay(reachable_areas_pst, how='difference', keep_geom_type=None, make_valid=True)
result_pst = calculate_population(morocco_shp, missing_areas_pst)

Processed 1000 points...
Processed 2000 points...
Processed 3000 points...
Failed point 3226: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2025-09-30T14:28:56Z', 'graph_version': '3', 'graph_date': '2025-11-10T13:42:44Z', 'osm_date': '2025-10-23T20:20:52Z', 'version': '9.4.0'}, 'timestamp': 1762783052159}})
Failed point 3229: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2025-09-30T14:28:56Z', 'graph_version': '3', 'graph_date': '2025-11-10T13:42:44Z', 'osm_date': '2025-10-23T20:20:52Z', 'version': '9.4.0'}, 'timestamp': 1762783052174}})
Failed point 3230: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2025-09-30T14:28:56Z', 'graph_version': '3', 'graph_date': '2025-11-10T13:42:44Z', 'osm_date': '2025-10-23T20:20:52Z', 'version': '9.4.0'}, 'timestamp': 1762783052178}})
Failed point 323

In [7]:
reachable_areas_pst = pd.concat([reachable_areas_ps,reachable_areas_t]).dissolve()
missing_areas_pst = morocco_shp.overlay(reachable_areas_pst, how='difference', keep_geom_type=None, make_valid=True)
result_pst = calculate_population(morocco_shp, missing_areas_pst)

In [8]:
result_pst

{'pop_total': 38043584.3125, 'pop_excluded': 7753534.375}

## Communes enclavées

In [9]:
# Find pixels outside of roads
missing_areas_pst = morocco_shp.overlay(reachable_areas_pst, how='difference', keep_geom_type=None, make_valid=True)
pop_path = os.path.join(data_path,'pop_morocco.tif')
if not os.path.exists(pop_path):
    create_population_map(pop_path)
pop_raster = rasterio.open(pop_path)
outside_mask, _ = mask(pop_raster, missing_areas_pst.geometry, crop=False, invert=False)
outside_mask = outside_mask[0]
outside_mask[outside_mask==-99999] = np.nan
profile = pop_raster.profile
profile.update(dtype=rasterio.float32, nodata=np.nan)

with rasterio.open('Data/pop_enclavee.tif', 'w', **profile) as dst:
    dst.write(outside_mask.astype(rasterio.float32), 1)



In [140]:
communes_shp = gpd.read_parquet(os.path.join(data_path,'communes_poverty.parquet'))
communes_shp = communes_shp.to_crs(epsg=4326)
communes_shp['Pop_WP'] = [0 if pd.isna(item['sum']) else item['sum'] for item in zonal_stats(communes_shp, pop_path, stats="sum",all_touched=True)]
communes_shp['Pop_enclavée'] = [0 if pd.isna(item['sum']) else item['sum'] for item in zonal_stats(communes_shp, os.path.join(data_path,'pop_enclavee.tif'), stats="sum",all_touched=True)]
communes_shp['Share_enclavée'] = communes_shp.Pop_enclavée/communes_shp.Pop_WP
communes_shp.to_parquet(os.path.join(data_path,'communes_enclavees.parquet'))

## Save plots

In [11]:
communes_shp = gpd.read_parquet(os.path.join(data_path,'communes_enclavees.parquet'))
viz_communes = create_map(communes_shp,0.5,"Commune",0.4)
viz_communes.save('output/enclavement_commune_50.html')
create_map(communes_shp,0.1,"Commune",0.2).save('output/enclavement_commune_10.html')
create_map(communes_shp,0.3,"Commune",0.2).save('output/enclavement_commune_30.html')


## Enclavement sur carreaux de 100km²

In [12]:
# Downsample population raster by a factor of 100
input_path_tot = os.path.join(data_path,"pop_morocco.tif")
output_path_tot = os.path.join(data_path,"pop_morocco_10km.tif")

downsample_raster(100,input_path_tot,output_path_tot)

# Downsample population enclavée raster by a factor of 100

input_path_enc = os.path.join(data_path,"pop_enclavee.tif")
output_path_enc = os.path.join(data_path,"pop_enclavee_10km.tif")

downsample_raster(100,input_path_enc,output_path_enc)

# Calculer le taux d'enclavement par pixel
ratio_enclavement_path = os.path.join(data_path,'Ratio_enclavement.tif')
compute_ratio_rasters(output_path_enc,output_path_tot,ratio_enclavement_path)

In [13]:
# Transform raster to vector for visualizations
with rasterio.open(ratio_enclavement_path) as src:
    image = src.read(1)  # read first band
    transform = src.transform

    # --- Correct mask: keep only finite (non-NaN) pixels
    mask = (image != src.nodata) & np.isfinite(image)

    # --- Polygonize raster
    results = (
        {'properties': {'Share_enclavée': float(v)}, 'geometry': s}
        for s, v in shapes(image, mask=mask, transform=transform)
    )
geoms = list(results)
gdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)
gdf = gdf[~pd.isna(gdf.Share_enclavée)]
gdf['id'] = gdf.index

viz_carreaux = create_map(gdf,0.5,"id",0.2)
viz_carreaux.save('output/enclavement_carreaux_50.html')
create_map(gdf,0.1,"id",0.2).save('output/enclavement_carreaux_10.html')
create_map(gdf,0.3,"id",0.2).save('output/enclavement_carreaux_30.html')